# Data Mining on BBC News Topics

## Introduction

Introduction here

In [1]:
import pandas as pd

**!** Name and describe the data set and include source: **2 marks**

Import only UK Domestic BBC News stories

In [2]:
allCc = pd.read_json('climate_change_data_subset.json')
allTrump = pd.read_json('trump_data_subset.json')
len(allCc)

1808

In [3]:
allCc.head()

,assetId,assetUri,body,headline,language,lastPublished,summary,title
0,38391034,/news/business-38391034,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",India's double first in climate battle,en-gb,2017-01-08T14:55:00+00:00,India opens two world-leading clean energy pro...,BBC News - India's double first in climate battle
1,38527710,/ukrainian/news-38527710,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",В Україну з півдня насуваються снігопади,uk,2017-01-06T06:31:56+00:00,"Через циклон, який насувається з Чорного моря,...",BBC Україна - В Україну з півдня насуваються с...
2,38600431,/news/uk-wales-38600431,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",£3m to protect Welsh and Irish coast from clim...,en-gb,2017-01-13T12:48:42+00:00,Coastal tourist sites most affected by climate...,BBC News - £3m to protect Welsh and Irish coas...
3,38745937,/news/science-environment-38745937,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Defining a true 'pre-industrial' climate period,en-gb,2017-01-25T23:11:25+00:00,Scientists suggest pushing back a baseline fro...,BBC News - Defining a true 'pre-industrial' cl...
4,39053678,/news/science-environment-39053678,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Most wood energy schemes are a 'disaster' for ...,en-gb,2017-04-23T14:02:03+00:00,A new report says that using wood pellets to g...,BBC News - Most wood energy schemes are a 'dis...


In [4]:
allTrump.head()

,assetId,assetUri,body,headline,language,lastPublished,summary,title
0,43597394,/portuguese/internacional-43597394,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",A pouco conhecida história de como os EUA leva...,pt-BR,2018-04-01T17:43:20+00:00,O governo americano foi o primeiro a violar o ...,BBC Brasil - A pouco conhecida história de com...
1,43607394,/portuguese/internacional-43607394,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",O ativista que criou uma identidade falsa para...,pt-BR,2018-04-02T07:34:34+00:00,Um ativista antirracismo passou um ano infiltr...,BBC Brasil - O ativista que criou uma identida...
2,43606088,/news/world-us-canada-43606088,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Donald Trump steps up attacks on Amazon,en-gb,2018-03-31T21:23:32+00:00,The president accuses the online retail giant ...,BBC News - Donald Trump steps up attacks on Am...
3,43601557,/news/world-us-canada-43601557,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",US may tie social media to visa applications,en-gb,2018-03-31T12:55:24+00:00,A state department proposal could require visa...,BBC News - US may tie social media to visa app...
4,43578462,/news/world-us-canada-43578462,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Is Trump ready for talks with North Korea?,en-gb,2018-03-30T21:20:28+00:00,What preparations is the White House making fo...,BBC News - Is Trump ready for talks with North...


## Data Cleaning

Watch out! Looks like my Climate Change data set in in date order whereas my Trump data set is in reverse date order!

**!** attribute and value domain discussion: **2 marks**

In [5]:
cc = allCc[allCc.assetUri.str.startswith('/news')]

In [6]:
len(cc)

664

In [7]:
trump = allTrump[allTrump.assetUri.str.startswith('/news')]

In [8]:
len(trump)

3254

In [9]:
trump.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3254 entries, 2 to 5133
Data columns (total 8 columns):
assetId          3254 non-null int64
assetUri         3254 non-null object
body             3250 non-null object
headline         3254 non-null object
language         3254 non-null object
lastPublished    3254 non-null object
summary          3254 non-null object
title            3254 non-null object
dtypes: int64(1), object(7)
memory usage: 228.8+ KB


In [10]:
cc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 664 entries, 0 to 1805
Data columns (total 8 columns):
assetId          664 non-null int64
assetUri         664 non-null object
body             662 non-null object
headline         664 non-null object
language         664 non-null object
lastPublished    664 non-null object
summary          664 non-null object
title            664 non-null object
dtypes: int64(1), object(7)
memory usage: 46.7+ KB


In [11]:
sum(trump['language'] != 'en-gb')

0

In [12]:
sum(cc['language'] != 'en-gb')

0

In [13]:
cc = cc.drop(columns=['assetId','language'])
trump = trump.drop(columns=['assetId','language'])

Two of the article bodies are missing, so these will be removed

In [14]:
cc = cc.dropna(axis=0, how='any')
trump = trump.dropna(axis=0, how='any')
cc.reset_index(drop=True)
trump.reset_index(drop=True);

Remove XML tags from the body

In [15]:
# from bs4 import BeautifulSoup as bs
# I started by using BeautifulSoup, however BeautifulSoup is for HTML whereas this is XML.
# The BBC has it's own XML standard which is not entirely conventional, 
# therefore I found it easier to write my own text extractor.

In [16]:
import re

def extract_text_into_new_column(df, column_name, new_column_name):
    temp_column = []
    for i in range(len(df)):
        text = df[column_name].iloc[i]
        tags = ["assetId", "assetTypeCode", "categoryId", "categoryName", 
                "crosshead", "changeQueueId", "embed", "hasShortForm", "language", 
                "provider", "publicationStatus", "workerCallingCard"]
        for tag in tags:
            tag_regex = '<'+ re.escape(tag) + '.*?' + re.escape(tag) + '>'
            text = re.sub(tag_regex, '', text)
        text = re.sub('<.*?>', '', text)
        timestamp_regex = '\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\+\d{2}:\d{2}'
        text = re.sub(timestamp_regex, '', text)
        text = re.sub('&amp;', '&', text)
        text = re.sub('\.\.\.', ' ', text)
        whitespace_after_full_stop_regex = '(?<=[a-z])\.(?=[A-Z])'
        text = re.sub(whitespace_after_full_stop_regex, '. ', text)
        whitespace_after_question_mark_regex = '(?<=[a-z])\?(?=[A-Z])'
        text = re.sub(whitespace_after_question_mark_regex, '. ', text)
        temp_column.append(text)
    df[new_column_name] = temp_column

extract_text_into_new_column(cc, 'body', 'cleanBody')
extract_text_into_new_column(trump, 'body', 'cleanBody')
trump.head()

,assetUri,body,headline,lastPublished,summary,title,cleanBody
2,/news/world-us-canada-43606088,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Donald Trump steps up attacks on Amazon,2018-03-31T21:23:32+00:00,The president accuses the online retail giant ...,BBC News - Donald Trump steps up attacks on Am...,President Donald Trump has stepped up his atta...
3,/news/world-us-canada-43601557,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",US may tie social media to visa applications,2018-03-31T12:55:24+00:00,A state department proposal could require visa...,BBC News - US may tie social media to visa app...,The Trump administration has said it wants to ...
4,/news/world-us-canada-43578462,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Is Trump ready for talks with North Korea?,2018-03-30T21:20:28+00:00,What preparations is the White House making fo...,BBC News - Is Trump ready for talks with North...,Never before has a US president prepared for a...
6,/news/world-us-canada-43591904,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Trump: Billions spent protecting other countri...,2018-03-29T19:31:50+00:00,Mr Trump says securing the Korean border comes...,BBC News - Trump: Billions spent protecting ot...,The US president says protecting the Korean bo...
7,/news/world-us-canada-43577444,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Trump loses bid to dismiss hotel lawsuit,2018-03-29T02:13:35+00:00,A judge allows a case to determine if the pres...,BBC News - Trump loses bid to dismiss hotel la...,Donald Trump's attempt to dismiss a lawsuit al...


In [17]:
trump.lastPublished = pd.to_datetime(trump.lastPublished)
cc.lastPublished = pd.to_datetime(cc.lastPublished)

Now all much of the data has been removed, the indexes need to be reset. The trump data (looking just by the head print out) appears to needs to be 'reversed' because the dates run in the opposite order. Both dataframes will be sorted in date order as they are reindexed because this is likely to make visualisation easier later. 

In [18]:
def sort_by_date_and_reindex(df, date_column):
    df = df.sort_values(by=[date_column])
    df = df.reset_index(drop=True)

sort_by_date_and_reindex(cc, 'lastPublished')
sort_by_date_and_reindex(trump, 'lastPublished')
trump.head()

,assetUri,body,headline,lastPublished,summary,title,cleanBody
2,/news/world-us-canada-43606088,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Donald Trump steps up attacks on Amazon,2018-03-31 21:23:32,The president accuses the online retail giant ...,BBC News - Donald Trump steps up attacks on Am...,President Donald Trump has stepped up his atta...
3,/news/world-us-canada-43601557,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",US may tie social media to visa applications,2018-03-31 12:55:24,A state department proposal could require visa...,BBC News - US may tie social media to visa app...,The Trump administration has said it wants to ...
4,/news/world-us-canada-43578462,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Is Trump ready for talks with North Korea?,2018-03-30 21:20:28,What preparations is the White House making fo...,BBC News - Is Trump ready for talks with North...,Never before has a US president prepared for a...
6,/news/world-us-canada-43591904,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Trump: Billions spent protecting other countri...,2018-03-29 19:31:50,Mr Trump says securing the Korean border comes...,BBC News - Trump: Billions spent protecting ot...,The US president says protecting the Korean bo...
7,/news/world-us-canada-43577444,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Trump loses bid to dismiss hotel lawsuit,2018-03-29 02:13:35,A judge allows a case to determine if the pres...,BBC News - Trump loses bid to dismiss hotel la...,Donald Trump's attempt to dismiss a lawsuit al...


## Visualization

In [19]:
import matplotlib.pyplot as plt

In [20]:
# Why this method?
# though we are splitting on an empty space to get the word count. This method is likely to give a good enough value
# because the paragraphs were marked with a <p> tag which has now been stripped out rather than a new line
# there are more accurate ways of obtaining a word count, however this is accurate enough for 
# the initial investigation that is this coursework

def create_column_containing_word_count(df, column_name, new_column_name):
    temp_column = []
    for i in range(len(df)):
        current_body = len(df[column_name].iloc[i].split(' '))
        temp_column.append(current_body)
    df[new_column_name] = temp_column
    
create_column_containing_word_count(cc, 'cleanBody', 'bodyLen')
create_column_containing_word_count(trump, 'cleanBody', 'bodyLen')
trump.head()

,assetUri,body,headline,lastPublished,summary,title,cleanBody,bodyLen
2,/news/world-us-canada-43606088,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Donald Trump steps up attacks on Amazon,2018-03-31 21:23:32,The president accuses the online retail giant ...,BBC News - Donald Trump steps up attacks on Am...,President Donald Trump has stepped up his atta...,320
3,/news/world-us-canada-43601557,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",US may tie social media to visa applications,2018-03-31 12:55:24,A state department proposal could require visa...,BBC News - US may tie social media to visa app...,The Trump administration has said it wants to ...,623
4,/news/world-us-canada-43578462,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Is Trump ready for talks with North Korea?,2018-03-30 21:20:28,What preparations is the White House making fo...,BBC News - Is Trump ready for talks with North...,Never before has a US president prepared for a...,1007
6,/news/world-us-canada-43591904,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Trump: Billions spent protecting other countri...,2018-03-29 19:31:50,Mr Trump says securing the Korean border comes...,BBC News - Trump: Billions spent protecting ot...,The US president says protecting the Korean bo...,20
7,/news/world-us-canada-43577444,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Trump loses bid to dismiss hotel lawsuit,2018-03-29 02:13:35,A judge allows a case to determine if the pres...,BBC News - Trump loses bid to dismiss hotel la...,Donald Trump's attempt to dismiss a lawsuit al...,330


In [21]:
cc.head()

,assetUri,body,headline,lastPublished,summary,title,cleanBody,bodyLen
0,/news/business-38391034,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",India's double first in climate battle,2017-01-08 14:55:00,India opens two world-leading clean energy pro...,BBC News - India's double first in climate battle,Two world-leading clean energy projects have o...,764
2,/news/uk-wales-38600431,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",£3m to protect Welsh and Irish coast from clim...,2017-01-13 12:48:42,Coastal tourist sites most affected by climate...,BBC News - £3m to protect Welsh and Irish coas...,Coastal tourist sites most affected by climate...,242
3,/news/science-environment-38745937,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Defining a true 'pre-industrial' climate period,2017-01-25 23:11:25,Scientists suggest pushing back a baseline fro...,BBC News - Defining a true 'pre-industrial' cl...,Scientists are seeking to define a new baselin...,822
4,/news/science-environment-39053678,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",Most wood energy schemes are a 'disaster' for ...,2017-04-23 14:02:03,A new report says that using wood pellets to g...,BBC News - Most wood energy schemes are a 'dis...,Using wood pellets to generate low-carbon elec...,991
8,/news/uk-39679584,"<body xmlns=""http://www.bbc.co.uk/asset"" xml:s...",March for Science: Thousands in London join gl...,2017-05-01 20:19:03,Organisers say the growth of fake news makes i...,BBC News - March for Science: Thousands in Lon...,Thousands of people have gathered in London to...,451


In [22]:
# plt.plot(cc.lastPublished)

In [23]:
# plt.plot(trump.lastPublished)

## Natural Language Processing

In [24]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/leives01/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

//TODO
* Tokenize each article into sentences. Each sentence is called a token. 
* Remove stop words -- stop words are very common words that don't really give any extra information regarding the meaning of the text ('a','as','and' etc.).
* Identify n-grams (bigrams)
* stemming similar words e.g. close, closing and closed 
* Part of Speech (POS) tagging


### Tokenizing the Text
I start by breaking the `cleanBody` into sentences and tokenizing them which means removing common stop words such as 'the', 'and', 'a' and 'that'. 

In [25]:
from nltk.tokenize import sent_tokenize

# sent = sent_tokenize(cc['cleanBody'].iloc[0])

# sent = "the cat sat on the mat. The cat did this. The cat did that."
# print(sent_tokenize(sent))

# matrix = []
# for i in range(len(cc)):
#     current = cc['cleanBody'].iloc[0]
#     sent = sent_tokenize(current)
#     matrix.append(sent)
    
# print(matrix)

def make_matrix_of_sentences(df, column):
    matrix = []
    for i in range(len(df)):
        current = df[column].iloc[i]
        sent = sent_tokenize(current)
        matrix.append(sent)
    return matrix

ccMatrix = make_matrix_of_sentences(cc, 'cleanBody')
trumpMatrix = make_matrix_of_sentences(trump, 'cleanBody')
# ccMatrix[0]

In [26]:
ccMatrix[100][0]

"Australia's Great Barrier Reef can be saved only if urgent steps are taken to reduce global warming, new research has warned."

In [27]:
cc['cleanBody'].iloc[100]

'Australia\'s Great Barrier Reef can be saved only if urgent steps are taken to reduce global warming, new research has warned. Attempting to stop coral bleaching through any other method will not be sufficient, according to scientists. The research, published in the journal Nature, said bleaching events should no longer be studied individually, but as threats to the reef\'s survival. The bleaching - or loss of algae - in 2016 was the worst on record."Climate change is the single greatest threat to the Great Barrier Reef," said co-author Prof Morgan Pratchett, from Queensland\'s James Cook University."It all comes down to what the governments in Australia and around the world do in terms of mitigating further rises in temperatures."     What causes coral bleaching?        Coral bleaching is caused by rising water temperatures resulting from two natural warm currents.    It is exacerbated by man-made climate change, as the oceans are absorbing about 93% of the increase in the Earth\'s h

In [28]:
# punkt includes stopwords in different languages
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
custom_stop_words = ["'s", "``", "''", "\""]
stop_words = set(stopwords.words('english') + list(punctuation) + list(custom_stop_words))

In [29]:
def tokenize_words(df, column):
    words_matrix = []
    for i in range (len(df)):
        current = df[column].iloc[i]
        these_words = [word for word in word_tokenize(current) if word not in stop_words]
        these_words
        words_matrix.append(these_words)
    return words_matrix

ccTokens = tokenize_words(cc, 'cleanBody')
trumpTokens = tokenize_words(trump, 'cleanBody')

In [30]:
print(trumpTokens[1010])

['President', 'Donald', 'Trump', "'Nambia", 'Africa', "'business", 'potential', 'Nambia', 'health', 'system', 'increasingly', 'self-sufficient', 'said', 'US', 'President', 'Donald', 'Trump', 'lunch', 'African', 'leaders', 'New', 'York', 'Wednesday', 'reeling', 'list', 'nations', 'achievements', 'But', 'country', 'exists', 'Could', 'US', 'leader', 'referring', 'Namibia', 'Zambia', 'Or', 'perhaps', 'The', 'Gambia', 'Social', 'media', 'users', 'wasted', 'time', 'offering', 'explanations', 'One', 'person', 'shared', 'image', 'US', 'race', 'activist', 'Rachel', 'Dolezal', 'identifies', 'black', 'despite', 'born', 'white', 'recently', 'visited', 'South', 'Africa', 'Find', 'Namibia', 'Zambia', 'The', 'Gambia', 'Namibia', 'country', 'profileNamibia', 'country', 'profileProvides', 'overview', 'Namibia', 'including', 'key', 'events', 'facts', 'sparsely', 'populated', 'stable', 'country', 'Africa', 'south-west', 'coast', 'BBC', 'News', 'Namibia', 'country', 'profileZambia', 'country', 'profileZam

In [52]:
from nltk.collocations import *

def get_bigrams(tokens_list):
    values = []
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    for i in range(len(tokens_list)):
        finder = BigramCollocationFinder.from_words(tokens_list[i])
        bigrams = list(finder.ngram_fd.items())
        bigrams.sort(key=lambda item: item[-1], reverse=True)
        values.append(bigrams)
    return values

trump_bigrams = get_bigrams(trumpTokens)
cc_bigrams = get_bigrams(ccTokens)
# trigram_finder = TrigramCollocationFinder.from_words(trumpTokens[1010])
# trump_trigrams = list(trigram_finder.ngram_fd.items())
# trump_trigrams.sort(key=lambda item: item[-1], reverse=True)
# trump_trigrams

In [73]:
def get_trigrams(tokens_list):
    values = []
    trigram_measures = nltk.collocations.TrigramAssocMeasures()
    for i in range(len(tokens_list)):
        finder = TrigramCollocationFinder.from_words(tokens_list[i])
        trigrams = list(finder.ngram_fd.items())
        trigrams.sort(key=lambda item: item[-1], reverse=True)
        values.append(trigrams)
    return values

trump_trigrams = get_trigrams(trumpTokens)
cc_trigrams = get_trigrams(ccTokens)

In [80]:
from nltk import pos_tag

def get_pos_tags(tokens_list):
    values = []
    for i in range(len(tokens_list)):
        current_list = tokens_list[i]
        values.append(nltk.pos_tag(current_list))
    return values

cc_pos_tags = get_pos_tags(ccTokens)
trump_pos_tags = get_pos_tags(trumpTokens)

In [81]:
new_trump_df = pd.DataFrame({
    'sentences': trumpMatrix, 
    'tokenized_words': trumpTokens,
    'bigrams': trump_bigrams,
    'trigrams': trump_trigrams,
    'pos_tags': trump_pos_tags
})

In [82]:
new_trump_df.head()

,bigrams,pos_tags,sentences,tokenized_words,trigrams
0,"[((Post, Office), 3), ((Mr, Trump), 3), ((atta...","[(President, NNP), (Donald, NNP), (Trump, NNP)...",[President Donald Trump has stepped up his att...,"[President, Donald, Trump, stepped, attacks, A...","[((US, Postal, Service), 2), ((US, Post, Offic..."
1,"[((social, media), 4), ((visa, applicants), 4)...","[(The, DT), (Trump, NNP), (administration, NN)...",[The Trump administration has said it wants to...,"[The, Trump, administration, said, wants, star...","[((New, York, Times), 2), ((Trump, 'in, crude)..."
2,"[((North, Korea), 12), ((Korea, talks), 5), ((...","[(Never, RB), (US, NNP), (president, NN), (pre...",[Never before has a US president prepared for ...,"[Never, US, president, prepared, summit, impor...","[((Who, going, represent), 3), ((going, repres..."
3,"[((The, US), 1), ((US, president), 1), ((presi...","[(The, DT), (US, NNP), (president, NN), (says,...",[The US president says protecting the Korean b...,"[The, US, president, says, protecting, Korean,...","[((The, US, president), 1), ((US, president, s..."
4,"[((Mr, Trump), 4), ((Trump, International), 3)...","[(Donald, NNP), (Trump, NNP), (attempt, NN), (...",[Donald Trump's attempt to dismiss a lawsuit a...,"[Donald, Trump, attempt, dismiss, lawsuit, all...","[((Trump, International, Hotel), 3), ((Lawyers..."


In [83]:
new_cc_df = pd.DataFrame({
    'sentences': ccMatrix, 
    'tokenized_words': ccTokens,
    'bigrams': cc_bigrams,
    'trigrams': cc_trigrams,
    'pos_tags': cc_pos_tags
})

In [84]:
new_cc_df.tail()

,bigrams,pos_tags,sentences,tokenized_words,trigrams
657,"[((away, coal), 4), ((climate, change), 4), ((...","[(The, DT), (UK, NNP), (Canada, NNP), (launche...",[The UK and Canada have launched a global alli...,"[The, UK, Canada, launched, global, alliance, ...","[((time, next, major), 2), ((next, major, UN),..."
658,"[((food, waste), 9), ((surplus, food), 5), ((B...","[(A, DT), (rising, VBG), (number, NN), (firms,...",[A rising number of firms are finding creative...,"[A, rising, number, firms, finding, creative, ...","[((issue, food, waste), 2), ((A, rising, numbe..."
659,"[((Meeting, carbon-reduction), 1), ((carbon-re...","[(Meeting, VBG), (carbon-reduction, JJ), (targ...",[Meeting carbon-reduction targets will be more...,"[Meeting, carbon-reduction, targets, challengi...","[((Meeting, carbon-reduction, targets), 1), ((..."
660,"[((More, hurricanes), 1), ((hurricanes, Rising...","[(More, JJR), (hurricanes, NNS), (Rising, VBG)...","[More hurricanes?, Rising temperatures?, BBC m...","[More, hurricanes, Rising, temperatures, BBC, ...","[((More, hurricanes, Rising), 1), ((hurricanes..."
661,"[((UK, US), 1), ((US, scientists), 1), ((scien...","[(UK, NNP), (US, NNP), (scientists, NNS), (lea...",[UK and US scientists will lead a five-year ef...,"[UK, US, scientists, lead, five-year, effort, ...","[((UK, US, scientists), 1), ((US, scientists, ..."


In [85]:
new_trump_df['sentences'].tail()

3245    [US President-elect Donald Trump says there mu...
3246    [Ford's decision to cancel a $1.6bn investment...
3247    [A leading US Republican says she fears for th...
3248    [2016 has been one dubbed for the history book...
3249    [Thirty-five Russian diplomats expelled from t...
Name: sentences, dtype: object